In [181]:
import pandas as pd
import numpy as np
from IPython.display import Image

## Ejercicio
FRA Existente: 91 x 182
<br>
FRA Inexistente: 91 x 220

In [182]:
# Queremos que pandas nos muestre ocho decimales al imprimir
pd.options.display.float_format = '{:,.8f}'.format

In [183]:
# Definimos variables
deuda = 4800000
tiie_91 = 0.047422
fra_ex = np.array([91, 182])
fra_inex = np.array([91, 220])

In [184]:
# Creamos tabla de tasas activas y pasivas
tasas = pd.DataFrame(index = fra_ex, columns = ['Pasivas', 'Activas'])
tasas['Pasivas'] = [((0.13/100) + tiie_91), ((0.292/100) + tiie_91)]
tasas['Activas'] = [((0.21/100) + tiie_91), ((0.396/100) + tiie_91)]
tasas

,Pasivas,Activas
91,0.04872200,0.04952200
182,0.05034200,0.05138200


In [185]:
# Llevamos tabla de tasas al plazo
t1 = tasas.iloc[0]*fra_ex[0]/360
t2 = tasas.iloc[1]*fra_ex[1]/360
tasas_plazo = pd.DataFrame([t1, t2])
tasas_plazo

,Pasivas,Activas
91,0.01231584,0.01251806
182,0.02545068,0.02597646


In [186]:
# Creamos tabla de resumen de tasas
index = ['Tasa Crédito', 'Tasa Inversión', 'Tasa Mercado']
columns = ['FRA 91', 'FRA 360', 'FRA 129']
resumen = pd.DataFrame(index = index, columns = columns)

resumen['FRA 91'].iloc[0] = ((1 + t2[1])/(1 + t1[0]) - 1)
resumen['FRA 91'].iloc[1] = ((1 + t2[0])/(1 + t1[1]) - 1)
tasa = 0.15
resumen['FRA 91'].iloc[2] = tasa*fra_ex[0]/360
resumen['FRA 360'].iloc[0] = resumen['FRA 91'].iloc[0]*360/fra_ex[0]
resumen['FRA 129'].iloc[0] = resumen['FRA 360'].iloc[0]*(fra_inex[1]-fra_inex[0])/360
resumen

,FRA 91,FRA 360,FRA 129
Tasa Crédito,0.01349442,0.05338452,0.01912945
Tasa Inversión,0.01277273,NaN,NaN
Tasa Mercado,0.03791667,NaN,NaN


In [187]:
# Queremos que pandas nos muestre dos decimales al imprimir
pd.options.display.float_format = '{:,.2f}'.format

In [188]:
# Creamos tabla de Capitalizaciones
index = ['Cap. Deuda a FRA 91',
        'Cap. Deuda a Mercado 91',
        'FRA-Mercado',
        'Fra-Mercado VA']
columns = ['Value']
capital = pd.DataFrame(index = index, columns = columns)

capital['Value'].iloc[0] = deuda*(1 + resumen['FRA 91'].iloc[0])
capital['Value'].iloc[1] = deuda*(1 + resumen['FRA 91'].iloc[2])
capital['Value'].iloc[2] = capital['Value'].iloc[1] - capital['Value'].iloc[0]
capital['Value'].iloc[3] = capital['Value'].iloc[2]/(1 + resumen['FRA 91'].iloc[2])
print(capital)

if capital['Value'].iloc[3] > 0:
    print("\nEsta cantidad se acredita al cliente")
else:
    print("\nEsta cantidad se acredita a la institución financiera")


                               Value
Cap. Deuda a FRA 91     4,864,773.22
Cap. Deuda a Mercado 91 4,982,000.00
FRA-Mercado               117,226.78
Fra-Mercado VA            112,944.31

Esta cantidad se acredita al cliente


In [189]:
# Creamos tabla de Liquidaciones y Conveniencia
index = ['Con Cobertura',
        'Sin Cobertura',
        'Conveniencia']
columns = ['Value']
conv = pd.DataFrame(index = index, columns = columns)

conv['Value'].iloc[0] = deuda*(1 + resumen['FRA 91'].iloc[0])
conv['Value'].iloc[1] = deuda*(1 + resumen['FRA 91'].iloc[2])
conv['Value'].iloc[2] = conv['Value'].iloc[1] - conv['Value'].iloc[0]
print(conv)

if conv['Value'].iloc[2] > 0:
    print("\nConviene la cobertura")
else:
    print("\nNo conviene la cobertura")

                     Value
Con Cobertura 4,864,773.22
Sin Cobertura 4,982,000.00
Conveniencia    117,226.78

Conviene la cobertura
